In [43]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_squared_error
import os

train_df = pd.read_csv('data/train_feats.csv')
#train_df.head()

dev_df = pd.read_csv('data/dev_feats.csv')
#dev_df.head()

test_df = pd.read_csv('data/val_feats.csv')
#test_df.head()

train_df.head(10)

,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,...,first_active_month_others,feature_1_2,feature_1_3,feature_1_5,feature_1_others,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1
0,382.0,278,37,25,7,-10.0,1.817983,8.0,307,7,...,1,0,0,0,1,1,0,0,1,0
1,39.0,879,29,149,20,-4.0,0.379389,5.0,2,20,...,1,1,0,0,0,1,0,0,1,0
2,188.0,705,33,226,16,-4.0,0.153180,16.0,80,37,...,1,0,0,0,1,1,0,0,1,0
3,19.0,307,19,57,5,-9.0,-0.342725,1.0,307,19,...,1,0,0,1,0,0,1,0,0,1
4,26.0,80,27,69,9,-3.0,-0.431381,8.0,80,27,...,1,1,0,0,0,1,0,0,1,0
5,101.0,307,19,17,22,-13.0,-0.255721,5.0,560,34,...,1,0,0,0,1,0,1,0,1,0
6,22.0,80,37,201,5,-3.0,-0.483944,7.0,80,27,...,1,0,1,0,0,1,0,0,0,1
7,138.0,705,33,116,9,-12.0,3.595754,1.0,606,17,...,1,0,1,0,0,0,0,1,0,1
8,249.0,80,37,69,9,-6.0,3.385383,10.0,80,34,...,1,0,0,0,1,1,0,0,1,0
9,236.0,705,33,117,23,-13.0,1501.922469,1.0,367,16,...,1,0,0,1,0,1,0,0,0,1


In [ ]:
test_params = {"max_depth": [50,75,100],
              "learning_rate" : [0.003,0.001,0.03,0.01],
              "num_leaves": [100,300,900,1200],
             }
lgbm_model = lgb.LGBMRegressor()
model = GridSearchCV(cv = 5, estimator = lgbm_model , param_grid = test_params)
model.fit(train_df.drop('target',axis=1), train_df['target'])

In [27]:
print(model.best_params_) 

{'learning_rate': 0.03, 'max_depth': 50, 'num_leaves': 100}


In [2]:
params = {}
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mse'
params['learning_rate'] = 0.03
params['max_depth'] = 50
params['num_leaves'] = 100

d_train = lgb.Dataset(train_df, train_df['target'])

#train
lgbm_model = lgb.train(params, d_train, 1000) #perform 1000 iterations

In [3]:
#predict for dev
target_pred = lgbm_model.predict(dev_df)

#results for dev
RMSE = np.sqrt(mean_squared_error(target_pred, dev_df['target']))
print("RMSE for dev dataset: ", RMSE)

RMSE for dev dataset:  0.14261033663093942


In [4]:
#predict for test
target_pred = lgbm_model.predict(test_df)

#results for test
RMSE = np.sqrt(mean_squared_error(target_pred, test_df['target']))
print("RMSE for test dataset: ", RMSE)

RMSE for test dataset:  0.15736749388987978


In [5]:
target_pred

array([ -0.77363057,  -2.29200255, -33.12437899, ...,  -0.11829376,
         0.53492113,   1.06305272])

<b>Preparing the submission for Kaggle</b>

In [ ]:
# from the feature_engineering_dataset

In [27]:
test_nealtered = pd.read_csv("data/test.csv")
card_id_column = test_nealtered["card_id"]
#test_nealtered
card_id_column

0         C_ID_0ab67a22ab
1         C_ID_130fd0cbdd
2         C_ID_b709037bc5
3         C_ID_d27d835a9f
4         C_ID_2b5e3df5c2
5         C_ID_5814b4f13c
6         C_ID_a1b3c75277
7         C_ID_f7cada36d3
8         C_ID_9d2bc8dfc4
9         C_ID_6d8dba8475
10        C_ID_4859ac9ed5
11        C_ID_0e281e8ad6
12        C_ID_8375dfdaa2
13        C_ID_79d149c008
14        C_ID_7f1041e8e1
15        C_ID_f72513589e
16        C_ID_a7f2d6984e
17        C_ID_15489b42d3
18        C_ID_e457bbd71d
19        C_ID_3f74e10c7e
20        C_ID_48243da9c4
21        C_ID_7ea1fcf117
22        C_ID_22e4a47c72
23        C_ID_5d30729184
24        C_ID_b54cfad8b2
25        C_ID_4b82e26faa
26        C_ID_60a453531a
27        C_ID_0c9315feec
28        C_ID_1e28804d36
29        C_ID_68de79f7e2
               ...       
123593    C_ID_cd42a5db89
123594    C_ID_715b9a183d
123595    C_ID_9b25df3309
123596    C_ID_043b3c2ccd
123597    C_ID_7685eae7b2
123598    C_ID_7ee1a21a95
123599    C_ID_2bf12fed6b
123600    C_

In [28]:
test = pd.read_csv("data/test_feats.csv")
test.head()

,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,new_most_frequent_city,new_most_frequent_state,new_min_month_lag,new_max_purchase_amount,first_active_month,feature_1,feature_2,feature_3
0,123.0,307.0,19.0,314.0,9.0,-5.0,0.768095,26.0,278.0,37.0,314.0,9.0,1.0,-0.078318,16.0,4,1,0
1,216.0,278.0,37.0,179.0,-1.0,-9.0,4.554145,31.0,278.0,37.0,69.0,9.0,1.0,3.129932,24.0,5,2,1
2,73.0,705.0,33.0,103.0,18.0,-13.0,-0.336684,3.0,605.0,2.0,69.0,9.0,1.0,-0.611669,41.0,5,1,1
3,30.0,560.0,34.0,233.0,9.0,-11.0,2.577843,1.0,432.0,27.0,23.0,9.0,2.0,-0.641872,25.0,2,2,0
4,70.0,705.0,33.0,344.0,18.0,-9.0,-0.097011,3.0,705.0,33.0,344.0,18.0,1.0,-0.667883,31.0,5,2,1


In [39]:
#predict for the actual test df
target_pred = lgbm_model.predict(test)
target_pred

array([-3.25678558e-05,  3.55715818e-04, -4.78166890e-05, ...,
       -6.29183995e-04, -6.29183995e-04, -6.67469597e-04])

In [45]:
df_results = pd.concat([card_id_column,pd.Series(target_pred, name = "target")], axis = 1)
df_results.shape

(123623, 2)

In [44]:
df_results.to_csv(os.path.join('data','lgbm_submission_1.csv'), index = False)